In [ ]:
import matplotlib.pyplot as plt
import pickle
import os
import numpy as np

# Calc FLOPS

In [ ]:
from calc_pseudo import clean, basic_flops
B_TARGET_PANEL_WIDTH = 48

In [ ]:
MAT_PATH = "./../../../example_mats_med/"
mat_paths = sum([[os.path.join(dir, file) for file in files] for dir, _, files in os.walk(MAT_PATH)], [])

In [ ]:
mat_flops = {}

In [ ]:
for mat_path in mat_paths:
    with open(mat_path) as f:
        test_mat = clean(np.loadtxt(f))
    FLOPS_PER_PANEL = basic_flops(test_mat, B_TARGET_PANEL_WIDTH)
    
    mat_flops[mat_path.split("./../../../")[1]] = FLOPS_PER_PANEL

## Load Data

In [ ]:
LOG_DIR = "./../../../logs/regpack_lm/pyfr/"

In [ ]:
# 'a_cols'
# 'a_nonzero'
# 'a_rows'
# 'a_size'
# 'a_unique'
# 'density'
# 'mat_file'
# 'speedup_avg'
# 'speedup_best'
# 'xsmm_local_avg'
# 'xsmm_local_best'
# 'xsmm_upstream_avg'
# 'xsmm_upstream_best'

with open(LOG_DIR+"run_1.dat", "rb") as f:
    run_1 = pickle.load(f)
with open(LOG_DIR+"run_2.dat", "rb") as f:
    run_2 = pickle.load(f)
with open(LOG_DIR+"run_3.dat", "rb") as f:
    run_3 = pickle.load(f)

## Sort Data

In [ ]:
def sort_values(x_term, run, mat_flops, t='best'):
    NUM_PANELS = 192000 / 48

    local_best = []
    ref_best = []
    local_best_x, local_best_y = [], []
    ref_best_x, ref_best_y = [], []

    for i, u in enumerate(run[x_term]):
        FLOPS_PER_PANEL = mat_flops[run['mat_file'][i]]

        time_per_panel_local = (run['xsmm_local_'+t][i]*1e-3)/NUM_PANELS
        time_per_panel_ref   = (run['xsmm_upstream_'+t][i]*1e-3)/NUM_PANELS
        
        local_best_x.append(u)
        local_best_y.append(FLOPS_PER_PANEL / time_per_panel_local)
        ref_best_x.append(u)
        ref_best_y.append(FLOPS_PER_PANEL / time_per_panel_ref)
    
    old_len = len(local_best_y)

    local_best_y = [x for _,x in sorted(zip(local_best_x,local_best_y))]
    local_best_x.sort()
    
    assert(old_len == len(local_best_y))

    ref_best_y = [x for _,x in sorted(zip(ref_best_x,ref_best_y))]
    ref_best_x.sort()

    return local_best_x, local_best_y, ref_best_x, ref_best_y

In [ ]:
x_terms = ['a_unique', 'a_cols', 'a_rows', 'a_size', 'density']
xlabels = ['Number of Unique Constants', 'Number of Columns',
           'Number of Rows', 'Size', 'Density']
xtitles = ['Number of Unique Constants in A', 'Number of Columns in A',
           'Number of Rows in A', 'Size of A', 'Density of A']

# Quad

In [ ]:
for i, x_term in enumerate(x_terms):
    # plot best
    plt.figure(figsize=(10,5))
    local_best_x1, local_best_y1, ref_best_x1, ref_best_y1 = \
        sort_values(x_term, run_1["quad"], mat_flops, 'best')
    local_best_x2, local_best_y2, ref_best_x2, ref_best_y2 = \
        sort_values(x_term, run_2["quad"], mat_flops, 'best')
    local_best_x3, local_best_y3, ref_best_x3, ref_best_y3 = \
        sort_values(x_term, run_3["quad"], mat_flops, 'best')
    local_best_y_avg = [sum(elem)/len(elem) for elem in zip(local_best_y1, local_best_y2, local_best_y3)]
    ref_best_y_avg = [sum(elem)/len(elem) for elem in zip(ref_best_y1, ref_best_y2, ref_best_y3)]

#     ln3_best_x1, local_best_y1, ref_best_x1, ref_best_y1 = \
#         sort_values(x_term, ln3_run_1["quad"], mat_flops, 'best')
#     local_best_x2, local_best_y2, ref_best_x2, ref_best_y2 = \
#         sort_values(x_term, ln3_run_2["quad"], mat_flops, 'best')
#     local_best_x3, local_best_y3, ref_best_x3, ref_best_y3 = \
#         sort_values(x_term, ln3_run_3["quad"], mat_flops, 'best')
#     ln3_best_y_avg = [sum(elem)/len(elem) for elem in zip(local_best_y1, local_best_y2, local_best_y3)]

#     plt.plot(ln3_best_x1, ln3_best_y_avg, label="RegPack 3 Acc", color="orange", marker=".")
    plt.plot(local_best_x1, local_best_y_avg, label="RegPack 2 Acc", color="limegreen", marker=".")
    plt.plot(ref_best_x1, ref_best_y_avg, label="RegPack Base", color="maroon", marker=".")

    plt.xlabel(xlabels[i])
    plt.ylabel("Pseudo-FLOP/s")
    plt.yscale("log", basey=10)
    plt.title("Quad: " + xtitles[i] + " vs Pseudo-FLOP/s")
    plt.ylim(top=10e9)
    plt.legend()
    plt.savefig("./../../../plots/regpack_lm/pyfr/quad/"+x_term+".pdf")
    plt.show()

# Hex

In [ ]:
for i, x_term in enumerate(x_terms):
    # plot best
    plt.figure(figsize=(10,5))
    local_best_x1, local_best_y1, ref_best_x1, ref_best_y1 = \
        sort_values(x_term, run_1["hex"], mat_flops, 'best')
    local_best_x2, local_best_y2, ref_best_x2, ref_best_y2 = \
        sort_values(x_term, run_2["hex"], mat_flops, 'best')
    local_best_x3, local_best_y3, ref_best_x3, ref_best_y3 = \
        sort_values(x_term, run_3["hex"], mat_flops, 'best')
    local_best_y_avg = [sum(elem)/len(elem) for elem in zip(local_best_y1, local_best_y2, local_best_y3)]
    ref_best_y_avg = [sum(elem)/len(elem) for elem in zip(ref_best_y1, ref_best_y2, ref_best_y3)]

#     ln3_best_x1, local_best_y1, ref_best_x1, ref_best_y1 = \
#         sort_values(x_term, ln3_run_1["hex"], mat_flops, 'best')
#     local_best_x2, local_best_y2, ref_best_x2, ref_best_y2 = \
#         sort_values(x_term, ln3_run_2["hex"], mat_flops, 'best')
#     local_best_x3, local_best_y3, ref_best_x3, ref_best_y3 = \
#         sort_values(x_term, ln3_run_3["hex"], mat_flops, 'best')
#     ln3_best_y_avg = [sum(elem)/len(elem) for elem in zip(local_best_y1, local_best_y2, local_best_y3)]

#     plt.plot(ln3_best_x1, ln3_best_y_avg, label="RegPack 3 Acc", color="orange", marker=".")
    plt.plot(local_best_x1, local_best_y_avg, label="RegPack 2 Acc", color="limegreen", marker=".")
    plt.plot(ref_best_x1, ref_best_y_avg, label="RegPack Base", color="maroon", marker=".")
    
    plt.xlabel(xlabels[i])
    plt.ylabel("Pseudo-FLOP/s")
    plt.yscale("log", basey=10)
    plt.ylim(1e8, 1e10)
    plt.title("Hex: " + xtitles[i] + " vs Pseudo-FLOP/s")
    plt.legend()
    plt.savefig("./../../../plots/regpack_lm/pyfr/hex/"+x_term+".pdf")
    plt.show()

# Tet

In [ ]:
for i, x_term in enumerate(x_terms):
    # plot best
    plt.figure(figsize=(10,5))
    local_best_x1, local_best_y1, ref_best_x1, ref_best_y1 = \
        sort_values(x_term, run_1["tet"], mat_flops, 'best')
    local_best_x2, local_best_y2, ref_best_x2, ref_best_y2 = \
        sort_values(x_term, run_2["tet"], mat_flops, 'best')
    local_best_x3, local_best_y3, ref_best_x3, ref_best_y3 = \
        sort_values(x_term, run_3["tet"], mat_flops, 'best')
    local_best_y_avg = [sum(elem)/len(elem) for elem in zip(local_best_y1, local_best_y2, local_best_y3)]
    ref_best_y_avg = [sum(elem)/len(elem) for elem in zip(ref_best_y1, ref_best_y2, ref_best_y3)]

#     ln3_best_x1, local_best_y1, ref_best_x1, ref_best_y1 = \
#         sort_values(x_term, ln3_run_1["tet"], mat_flops, 'best')
#     local_best_x2, local_best_y2, ref_best_x2, ref_best_y2 = \
#         sort_values(x_term, ln3_run_2["tet"], mat_flops, 'best')
#     local_best_x3, local_best_y3, ref_best_x3, ref_best_y3 = \
#         sort_values(x_term, ln3_run_3["tet"], mat_flops, 'best')
#     ln3_best_y_avg = [sum(elem)/len(elem) for elem in zip(local_best_y1, local_best_y2, local_best_y3)]

#     plt.plot(ln3_best_x1, ln3_best_y_avg, label="RegPack 3 Acc", color="orange", marker=".")
    plt.plot(local_best_x1, local_best_y_avg, label="RegPack 2 Acc", color="limegreen", marker=".")
    plt.plot(ref_best_x1, ref_best_y_avg, label="RegPack Base", color="maroon", marker=".")

    plt.xlabel(xlabels[i])
    plt.ylabel("Pseudo-FLOP/s")
    plt.yscale("log", basey=10)
    plt.ylim(5e9,3e10)
    plt.title("Tet: " + xtitles[i] + " vs Pseudo-FLOP/s")
    plt.legend()
    plt.savefig("./../../../plots/regpack_lm/pyfr/tet/"+x_term+".pdf")
    plt.show()

# Tri

In [ ]:
for i, x_term in enumerate(x_terms):
    # plot best
    plt.figure(figsize=(10,5))
    local_best_x1, local_best_y1, ref_best_x1, ref_best_y1 = \
        sort_values(x_term, run_1["tri"], mat_flops, 'best')
    local_best_x2, local_best_y2, ref_best_x2, ref_best_y2 = \
        sort_values(x_term, run_2["tri"], mat_flops, 'best')
    local_best_x3, local_best_y3, ref_best_x3, ref_best_y3 = \
        sort_values(x_term, run_3["tri"], mat_flops, 'best')
    local_best_y_avg = [sum(elem)/len(elem) for elem in zip(local_best_y1, local_best_y2, local_best_y3)]
    ref_best_y_avg = [sum(elem)/len(elem) for elem in zip(ref_best_y1, ref_best_y2, ref_best_y3)]

#     ln3_best_x1, local_best_y1, ref_best_x1, ref_best_y1 = \
#         sort_values(x_term, ln3_run_1["tri"], mat_flops, 'best')
#     local_best_x2, local_best_y2, ref_best_x2, ref_best_y2 = \
#         sort_values(x_term, ln3_run_2["tri"], mat_flops, 'best')
#     local_best_x3, local_best_y3, ref_best_x3, ref_best_y3 = \
#         sort_values(x_term, ln3_run_3["tri"], mat_flops, 'best')
#     ln3_best_y_avg = [sum(elem)/len(elem) for elem in zip(local_best_y1, local_best_y2, local_best_y3)]

#     plt.plot(ln3_best_x1, ln3_best_y_avg, label="RegPack 3 Acc", color="orange", marker=".")
    plt.plot(local_best_x1, local_best_y_avg, label="RegPack 2 Acc", color="limegreen", marker=".")
    plt.plot(ref_best_x1, ref_best_y_avg, label="RegPack Base", color="maroon", marker=".")

    plt.xlabel(xlabels[i])
    plt.ylabel("Pseudo-FLOP/s")
    plt.yscale("log", basey=10)
    plt.title("Tri: " + xtitles[i] + " vs Pseudo-FLOP/s")
    plt.legend()
    plt.savefig("./../../../plots/regpack_lm/pyfr/tri/"+x_term+".pdf")
    plt.show()